# Interactive Database Mapping #

This Notebook contains a code to create an interactive and searchable map database of onshore wind turbine projects across the UK. 

The Notebook is split into two main sections:
- Part 1: Data Preparation
- Part 2: Making an Interactive Map

## Part 1: Data Preparation ##

We will begin by importing the neccessary packages for the analysis and map making:

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search
import ipyleaflet
import json
import matplotlib.pyplot as plt
from matplotlib import colormaps
import matplotlib.colors as mcolors

#### Clean the raw data to prepare for map making: ####

##### 1. Load in the Rewnewable Energy Planning Database (REPD) quarterly extract for January 2025*: 

The data is currently in csv format.

*Note: As long as the data columns are the same, the following code should be compatible with future quarterly extracts, allowing maintainance of the database.

In [ ]:
# Load with cp1252 to avoid Unicode error as file is not in UTF-8 encoding.
repd_data = pd.read_csv('repd-q4-jan-2025.csv', encoding='cp1252')

# Then normalize any weird characters to ensure file reads correctly.
repd_data = repd_data.map(lambda x: x.replace('\xa0', ' ') if isinstance(x, str) else x)

repd_data.head() # Show raw data to get familiar with the content.

##### 2. Filter the REPD dataset to focus only on onshore wind turbine projects: #####

In [ ]:
repd_data = repd_data[repd_data['Technology Type'] == 'Wind Onshore']

##### 3. Filter out any unneccessary data columns: ##### 
(e.g. data related to other technologies such as solar, reference numbers used only by REPD data, etc..)

In [ ]:
# 3.1. First get a list of all the available data columns and their indices to help identify useful information:
index_columns = [(index, column) for index, column in enumerate(repd_data.columns)]

# Print the list
for index, column in index_columns:
    print(f"{index}: {column}")

In [ ]:
# 3.2. Now filter the columns by dropping irrelevant columns by name:
repd_wind = repd_data.drop(columns=['Old Ref ID', 'Ref ID', 'Technology Type', 
                                    'Storage Type', 'CHP Enabled', 'Storage Co-location REPD Ref ID', 
                                    'Share Community Scheme', 'CfD Allocation Round', 'RO Banding (ROC/MWh)', 
                                    'CfD Capacity (MW)', 'Mounting Type for Solar', 'Are they re-applying (New REPD Ref)', 
                                    'Are they re-applying (Old REPD Ref) ', 'Development Status', 'Offshore Wind Round', 
                                    'Heat Network Ref', 'Solar Site Area (sqm)'])

repd_wind.head() # View updated repd_wind dataset

##### 4. Perform checks and cleaning operations on the remaining data to ensure full compatibility with mapping: #####

In [ ]:
# 4.1. Check for suspicious characters in remaining columns which may stop the map from generating correctly:
for col in repd_wind.columns:
    if repd_wind[col].dtype == object:
        if repd_wind[col].str.contains(r'\\[0-9]').any():
            print(f"Suspicious escape sequence in column: {col}")

In [ ]:
# 4.1.2. Fix suspicious characters by escaping backslashes in all string/object columns:
for col in repd_wind.select_dtypes(include='object').columns:
    repd_wind[col] = repd_wind[col].str.replace(r'\\', r'\\\\', regex=True)

In [ ]:
# 4.2. Check for non-numeric entries in the coordinate columns and drop these data rows:
non_numeric_x = repd_wind[~repd_wind['X-coordinate'].apply(pd.to_numeric, errors='coerce').notnull()]
non_numeric_y = repd_wind[~repd_wind['Y-coordinate'].apply(pd.to_numeric, errors='coerce').notnull()]

# Display rows with non-numeric coordinates
print("Non-numeric X-coordinates:")
print(non_numeric_x[['X-coordinate', 'Y-coordinate']])

print("Non-numeric Y-coordinates:")
print(non_numeric_y[['X-coordinate', 'Y-coordinate']])

# Drop rows with non-numeric coordinates
repd_wind = repd_wind.dropna(subset=['X-coordinate', 'Y-coordinate'])

##### 5. Convert the tabular data into a GeoDataFrame using the 'X-coordinate' and 'Y-coordinate' columns:

In [ ]:
# Create the GeoDataFrame
wind_turbines_27700 = gpd.GeoDataFrame(repd_wind,
    geometry=gpd.points_from_xy(repd_wind['X-coordinate'], repd_wind['Y-coordinate']), # Create geometry from X, Y coordinate columns
    crs='epsg:27700') # Set ESPG to British National Grid

wind_turbines_27700.head() # Check if successful

Folium is most compatible with geographic coordinate systems, such as WGS 84 (epsg:4326). We will make a copy of the GeoDataFrame with WGS 84 coordinates for use in the folium map. 
We will also keep a copy with the British National Grid coordinates as this would be more useful for precise measurements of distances or any other analyses of interest.

In [ ]:
# Make a copy of the GeoDataFrame with WGS 84:
wind_turbines_4326 = wind_turbines.to_crs(epsg=4326)
print(wind_turbines_4326.crs) # Check if successful

# Drop the now unneccessary coordinate columns:
wind_turbines_4326 = wind_turbines_4326.drop(columns=['X-coordinate', 'Y-coordinate'])

For the purposes of the interactive database, it may be more useful to replace NaN values with N/A values to signify to the user that this information is not available from the REPD dataset, but may be available elsewhere.

In [ ]:
# Change NaN values for string columns to string N/A values:
str_columns = wind_turbines_4326.select_dtypes(include=['object'])

# Fill NaN values in string columns with 'N/A'
wind_turbines_4326[str_columns.columns] = str_columns.fillna('N/A')
wind_turbines_4326

##### 6. Perform last checks for invalid coordinates and invalid geometries, and drop any invalid results:

In [ ]:
# 6.1. Check for any data outside valid longitude (-180 to 180) or latitude (-90 to 90):
invalid_coords = wind_turbines_4326[
    (wind_turbines_4326.geometry.x < -180) |
    (wind_turbines_4326.geometry.x > 180) |
    (wind_turbines_4326.geometry.y < -90) |
    (wind_turbines_4326.geometry.y > 90)]

print(f"Out-of-bounds coordinates: {len(invalid_coords)}")
invalid_coords[['Site Name', 'geometry']]

In [ ]:
# 6.2. Check for invalid geometries:
invalid_geom = wind_turbines_4326[
    wind_turbines_4326.geometry.is_empty | wind_turbines_4326.geometry.isna()]

print(f"Invalid geometries: {len(invalid_geom)}")
invalid_geom[['Site Name', 'geometry']]

In [ ]:
# 6.3. Filter out invalid geometries and coordinates:
wind_turbines_4326 = wind_turbines_4326[
    wind_turbines_4326.geometry.notnull() &
    wind_turbines_4326.geometry.is_valid &
    wind_turbines_4326.geometry.apply(lambda geom: geom.is_empty == False) &
    wind_turbines_4326.geometry.apply(lambda geom: geom.x != float("inf") and geom.y != float("inf"))]

Now that the data is fully cleaned and compatible with Folium, we will save a copy of the data to GeoJSON format to ensure compatibility with the Folium Search plugin. This will allow map users to search the map based on any keyword from the dataset.

In [ ]:
# Save cleaned-up GeoDataBase as geopackage (to preserve column name lengths):
wind_turbines_4326.to_file('turbines_shp.gpkg', driver='GPKG')

# Save GeoJSON copy for compatibility with search plugin.

# First we will combine all fields into a new column which will be used in the search function. 
# This will allow search based on any column in the dataset rather than limit to a specific column:
wind_turbines_4326['search_all'] = wind_turbines_4326.apply(
    lambda row: " | ".join([str(val) for val in row.values]), axis=1)

wind_turbines_4326.to_file('turbines_json.geojson', driver='GeoJSON')



## Part 2: Making an Interactive Map

First read in the newly saved files:

In [ ]:
turbines_shp = gpd.read_file()
turbines_json = gpd.read_file('../DataPrep/wind_turbines.geojson')

##### 1. Define Functions for use in map making steps:

In [ ]:
# Define functions for later use

# modify pop-up settings to only show relevant information, i.e. if N/A, dont show
# but only for columns related to planning process i.e. from planning application withdrawn column to operational column (last column)
# Function to create conditional popups, excluding N/A values for columns 15+ 
#(columns covering different stages in planning process, info unneccessary for all)
def conditional_popups(gdf):
    """
    Returns a list of popup HTML strings for all rows in a GeoDataFrame.
    """
    popups = []
    for _, row in gdf.iterrows():
        popup_cond = []

        for column in row.index[:16]:
            popup_cond.append(f"<b>{column}:</b> {row[column]}")

        for column in row.index[16:]:
            value = str(row[column])
            if value != 'N/A':
                popup_cond.append(f"<b>{column}:</b> {value}")

        popups.append("<br>".join(popup_cond))
    return popups

# Function to assign marker radius based on turbine height bins
def marker_radius(height):
    try:
        height = float(height)
        if height < 50:
            return 4
        elif height < 100:
            return 7
        elif height < 150:
            return 10
        else:
            return 13
    except:
        return 2  # default if missing/invalid

# Function to assign marker colour based on a specific column
def marker_colour(gdf, column_name):
    """
    Generate a dictionary of colors for each unique category in a given column.
    """
    unique_values = gdf[column_name].dropna().unique()
    n = len(unique_values)

    # Use a colormap from matplotlib
    cmap = colormaps.get_cmap('tab20').resampled(n)
    colors = [mcolors.to_hex(cmap(i)) for i in range(n)]

    # Map each unique category to a color
    return dict(zip(unique_values, colors))

# Define a legend function based on the colours and sizes defined previously
def add_legend(map, colour_dict):
    """
    Add a custom HTML legend to the map, including both color and size information.
    """
    size_dict = {
        '0-50 meters': 4,
        '50-100 meters': 7,
        '100-150 meters': 10,
        '150+ meters': 13
    }

    # Start building the legend HTML
    legend_html = '''
        <div style="position: fixed; 
                    bottom: 50px; left: 50px; width: 250px; height: auto; 
                    border:2px solid grey; z-index:9999; font-size:14px; 
                    background-color: white; opacity: 0.7; padding: 10px;">
                    <b>Wind Turbine Legend</b><br>
                    <b>Development Status (Short)</b><br>
    '''
    
    # Add each category and its color to the legend
    for category, color in colour_dict.items():
        legend_html += f'<i style="background: {color}; width: 20px; height: 20px; display: inline-block; margin-right: 8px;"></i>{category}<br>'
    
    # Add turbine height sizing information to the legend
    legend_html += '<br><b>Turbine Height (m)</b><br>'
    for height_range, radius in size_dict.items():
        # Add circular markers in the legend that match the size of the radius
        legend_html += f'<i style="background: gray; border-radius: 50%; width: {radius * 2}px; height: {radius * 2}px; display: inline-block; margin-right: 8px;"></i>{height_range}<br>'
  
    legend_html += '</div>'
    
    # Add the legend to the map as a popup
    map.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
# Get the list of columns with their index numbers
index_columns = [(index, column) for index, column in enumerate(wind_turbines.columns)]

# Print the list
for index, column in index_columns:
    print(f"{index}: {column}")


In [ ]:
unique_values = list(wind_turbines["Development Status (short)"].dropna().unique())
unique_values

In [ ]:
# add additional raster and vector data for viewing context

In [ ]:
#TEST
# Create the map centered around the turbines' mean coordinates
m = folium.Map(location=[
    wind_turbines.geometry.y.mean(),
    wind_turbines.geometry.x.mean()
], zoom_start=6)

# Generate color mapping based on 'Development Status (short)' column
colours = marker_colour(wind_turbines, 'Development Status (short)')

# Add CircleMarkers for wind turbines
for i, (idx, row) in enumerate(wind_turbines.iterrows()):
    popup_html = f"<b>{row['Site Name']}</b><br><b>Status:</b> {row['Development Status (short)']}"
    status = row['Development Status (short)']
    color = colours.get(status, 'black')
    radius = 2  # Default radius if not specified

    if pd.notna(row.geometry.y) and pd.notna(row.geometry.x):
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_html, max_width=300)
        ).add_to(m)

# Create GeoJSON layer for wind turbines (only used for search functionality)
geojson_layer = folium.GeoJson(
    wind_turbines,  # Directly use the GeoDataFrame
    style_function=lambda feature: {
        "color": "transparent",  # Make the border fully transparent
        "fillColor": "transparent",  # Make the fill fully transparent
        "fillOpacity": 0,  # Ensure it is fully transparent
        "opacity": 0  # Ensure it is fully transparent
    },
    control=False,  # Don't include this in the LayerControl
    visible=False  # Ensure it's not visible on the map
).add_to(m)

# Add Search plugin for the GeoJSON layer (for search functionality only)
search = Search(
    layer=geojson_layer,  # Pass the invisible GeoJSON layer to the search function
    geom_type="Point",
    search_label="search_all",  # Use the combined column for search
    placeholder="Search anything...",
    collapsed=False
).add_to(m)

# Add satellite basemap
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=True,
    control=True  # Enable layer control for the satellite basemap
).add_to(m)

# Add Layer Control to allow users to toggle satellite layers
folium.LayerControl().add_to(m)

# Add the custom legend
add_legend(m, colours)

# Display the map
m